In [1]:
!pip install sagemaker boto3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 118.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.0/426.0 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
import os, boto3, sagemaker, zipfile
import shutil

AWS_ACCESS_KEY_ID = ""
AWS_SECRET_ACCESS_KEY = "i"
AWS_DEFAULT_REGION = "us-east-1"  # Change if needed

session = boto3.session.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_DEFAULT_REGION
)

sm_session = sagemaker.Session(boto_session=session)
role = "arn:aws:iam::058264390933:role/Dhanu@08"  # IAM role with S3 + SageMaker permissions
bucket = "my-sagemaker-models1"



In [ ]:
model_dir = "/content/drive/MyDrive/tinyllama-it-support-merged"
output_path = "/content/tinyllama-it-support-lora.tar.gz"

shutil.make_archive(output_path.replace(".tar.gz", ""), 'gztar', model_dir)
print("✅ Archive created:", output_path)

# ================================
# 3. Upload to S3
# ================================
s3 = session.client("s3")   
s3_key = "tinyllama-it-support-merged.tar.gz"

s3.upload_file(output_path, bucket, s3_key)

model_artifact = f"s3://{bucket}/{s3_key}"
print("✅ Uploaded to:", model_artifact)

# ================================
# 4. Deploy on SageMaker
# ================================
huggingface_model = HuggingFaceModel(
    model_data=model_artifact,
    role=role,
    transformers_version="4.37",
    pytorch_version="2.1",
    py_version="py310",
    sagemaker_session=sm_session
)

endpoint_name = "tinyllama-it-support-endpoint-merged"

predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",   # for demo
    endpoint_name=endpoint_name
)

print("✅ Deployed endpoint:", endpoint_name)

✅ Archive created: /content/tinyllama-it-support-lora.tar.gz
✅ Uploaded to: s3://my-sagemaker-models1/tinyllama-it-support-merged.tar.gz
-------!✅ Deployed endpoint: tinyllama-it-support-endpoint-merged
